In [36]:
import pandas as pd

In [37]:
raw_data = pd.read_csv("./data/data.csv", skiprows=[2])

In [38]:
raw_data = raw_data[raw_data["Status"] == "IP Address"]

In [39]:
raw_data.drop(
    columns=[
        "Status",
        "IPAddress",
        "RecipientLastName",
        "RecipientFirstName",
        "RecipientEmail",
        "ExternalReference",
        "LocationLatitude",
        "LocationLongitude",
        "DistributionChannel",
        "UserLanguage",
    ]
)

,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,Q_RecaptchaScore,QC1,QD1,...,Q74,Q79,ST6_First Click,ST6_Last Click,ST6_Page Submit,ST6_Click Count,QS11,Q52,Q80,Q81
19,2024-01-16 05:19:14,2024-01-16 05:33:55,100,881,True,2024-01-16 05:33:56,R_72uxJTbT26KJZSJ,1,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2024-01-16 05:57:58,2024-01-16 06:07:10,100,551,True,2024-01-16 06:07:11,R_1xlLKSLVgimtnPj,0.8999999761581421,Yes,Male,...,Somewhat confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,2024-01-16 05:58:26,2024-01-16 06:08:48,100,622,True,2024-01-16 06:08:49,R_6nZIlRrs9aUGKHL,1,Yes,Male,...,Neither confident or unsure,Dissatisfied,NaN,NaN,NaN,NaN,3rd of November,8th of November,Not confident at all,Very dissatisfied
22,2024-01-16 06:31:46,2024-01-16 06:44:03,100,736,True,2024-01-16 06:44:03,R_7PplWzAG0igeOml,1,Yes,Male,...,Somewhat confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN,10th of November,24th of November,Somewhat confident,Neither dissatisfied or satisfied
23,2024-01-16 05:21:40,2024-01-16 08:17:14,100,10534,True,2024-01-16 08:17:15,R_2nipTYf8mhH2Yus,0.8999999761581421,Yes,Non-binary / third gender,...,Somewhat confident,Satisfied,NaN,NaN,NaN,NaN,10th of November,24th of November,Somewhat confident,Satisfied
24,2024-01-16 09:26:32,2024-01-16 09:34:10,100,457,True,2024-01-16 09:34:11,R_1dy2jAJUx1DgK6R,0.800000011920929,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2024-01-16 09:27:48,2024-01-16 09:36:15,100,506,True,2024-01-16 09:36:15,R_2esfOQZiXvHpgod,0.8999999761581421,Yes,Male,...,Somewhat confident,Satisfied,NaN,NaN,NaN,NaN,10th of November,29th of November,Neither confident or unsure,Dissatisfied
26,2024-01-17 03:09:03,2024-01-17 03:19:06,100,602,True,2024-01-17 03:19:06,R_2z5zMoIKEHtkx0S,0.800000011920929,Yes,Female,...,Not confident,Neither dissatisfied or satisfied,NaN,NaN,NaN,NaN,10th of November,24th of November,Confident,Very satisfied
27,2024-01-17 03:25:49,2024-01-17 03:28:42,100,173,True,2024-01-17 03:28:43,R_6QE4FQzYCk4GaNV,1,Yes,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2024-01-17 03:25:40,2024-01-17 03:33:06,100,446,True,2024-01-17 03:33:06,R_5IGX6rHpxzyVRGc,0.8999999761581421,Yes,Female,...,Not confident at all,Very dissatisfied,NaN,NaN,NaN,NaN,10th of November,24th of November,Not confident at all,Dissatisfied


In [40]:
raw_data.to_csv("./data/data3.csv")

In [41]:
demographics = raw_data.loc[:, :"Q_RecaptchaScore"]
# replaces Likert scale with numbers
replace_map = {
    "Satisfied": 4,
    "Very satisfied": 5,
    "Neither dissatisfied or satisfied": 3,
    "Dissatisfied": 2,
    "Very dissatisfied": 1,
    "Confident": 4,
    "Somewhat confident": 5,
    "Neither confident or unsure": 3,
    "Not confident": 2,
    "Not confident at all": 1,
}
raw_data.replace(replace_map, inplace=True)

# Make Score for User satisfaction and User Confidence
# raw_data["US"] = 0
# raw_data["UC"] = 0

complex_responses = raw_data.loc[:, "QC1.1":"Q68"]
simple_responses = raw_data.loc[:, "QS1":]

In [1440]:
# simple_responses.to_csv("./data/simple.csv")
# complex_responses.to_csv("./data/complex.csv")

In [42]:
# Dropping time metrics as we do not have enough yet
# Dropping time metrics as we do not have enough yet
columns_to_drop = (
    list(range(5, 9)) + list(range(17, 21)) + list(range(28, 32)) + list(range(37, 41))
)

# print(simple_responses.columns)
simple_responses.drop(simple_responses.columns[columns_to_drop], axis=1, inplace=True)
complex_responses.drop(complex_responses.columns[columns_to_drop], axis=1, inplace=True)

simple_responses = simple_responses.dropna(thresh=10, axis=0)
complex_responses = complex_responses.dropna(thresh=20, axis=0)

simple_responses.to_csv("./data/simple.csv")
complex_responses.to_csv("./data/complex.csv")

# print(complex_responses.columns)

simple_responses.head(20)

,QS1,QS2,QS3,Q73,Q72,QS4_1,QS4_2,QS4_3,QS4_4,QS4_5,...,Q77,QS8,QS9,QS10,Q74,Q79,QS11,Q52,Q80,Q81
20,False,False,True,3.0,3.0,1,5,4,3,2,...,4.0,Q2,October,January-February,5.0,3.0,NaN,NaN,NaN,NaN
21,True,False,False,5.0,2.0,1,5,4,3,2,...,5.0,Q2,February,November-December,3.0,2.0,3rd of November,8th of November,1.0,1.0
22,False,False,True,5.0,2.0,1,4,5,3,2,...,1.0,Q2,February,November-December,5.0,3.0,10th of November,24th of November,5.0,3.0
23,False,False,False,5.0,3.0,NaN,NaN,NaN,NaN,NaN,...,5.0,Q2,February,November-December,5.0,4.0,10th of November,24th of November,5.0,4.0
25,False,False,True,5.0,4.0,1,5,4,3,2,...,1.0,Q1,October,January-February,5.0,4.0,10th of November,29th of November,3.0,2.0
26,False,False,False,1.0,2.0,1,5,4,3,2,...,5.0,Q2,February,January-February,2.0,3.0,10th of November,24th of November,4.0,5.0
28,True,False,False,2.0,2.0,1,5,4,3,2,...,2.0,Q3,October,November-December,1.0,1.0,10th of November,24th of November,1.0,2.0
29,False,False,True,5.0,4.0,1,5,4,3,2,...,4.0,Q1,October,January-February,5.0,2.0,10th of November,29th of November,3.0,2.0
32,False,False,True,5.0,3.0,1,5,4,3,2,...,3.0,Q2,October,January-February,5.0,4.0,10th of November,24th of November,5.0,4.0
34,False,False,True,5.0,3.0,1,5,4,3,2,...,3.0,Q2,February,January-February,5.0,2.0,10th of November,24th of November,4.0,4.0


In [43]:
answer_sheet = pd.read_excel("./data/answer_sheet.xlsx")
answer_sheet = answer_sheet.astype(str)

In [44]:
answer_sheet.head()

,Q1.1,Q1.2,Q1.3,Q2.1,Q2.2,Q2.3,Q2.4,Q2.5,Q2.6,Q3.1,Q3.2,Q3.3,Q3.4,Q3.5,Q4.1,Q4.2,Q4.3,Q5.1,Q5.2
0,False,False,True,5,1,4,3,2,34,Executive,2,1,3,4,Q2,Febuary,January-Febuary,10th of November,24th of November


In [45]:
def calculate(question, answer, question_nr):
    score_list = []

    for _, row in question.iterrows():
        correct_answers = (row == answer.iloc[0, :]).sum()
        total_questions = len(row)
        accuracy = correct_answers / total_questions

        score_list.append({question_nr + "_Accuracy": accuracy})

    score_df = pd.DataFrame(score_list)
    return score_df

In [46]:
def calculate_scores(responses, answer_sheet, complexity):
    # Retrieve answers from answer sheet
    answer_Q1 = answer_sheet.iloc[:, 0:3]
    answer_Q2 = answer_sheet.iloc[:, 3:9]
    answer_Q3 = answer_sheet.iloc[:, 9:14]
    answer_Q4 = answer_sheet.iloc[:, 14:17]
    answer_Q5 = answer_sheet.iloc[:, 17:]

    # Retreive questions and corresponding likert responses
    question_Q1 = responses.iloc[:, 0:3]
    question_Q1.columns = ["Q1.1", "Q1.2", "Q1.3"]
    likert_Q1 = responses.iloc[:, 3:5]
    likert_Q1.columns = ["Q1_Con", "Q1_Sat"]

    question_Q2 = responses.iloc[:, 5:11]
    question_Q2.columns = ["Q2.1", "Q2.2", "Q2.3", "Q2.4", "Q2.5", "Q2.6"]
    likert_Q2 = responses.iloc[:, 11:13]
    likert_Q2.columns = ["Q2_Con", "Q2_Sat"]

    question_Q3 = responses.iloc[:, 13:18]
    question_Q3.columns = ["Q3.1", "Q3.2", "Q3.3", "Q3.4", "Q3.5"]
    likert_Q3 = responses.iloc[:, 18:20]
    likert_Q3.columns = ["Q3_Con", "Q3_Sat"]

    question_Q4 = responses.iloc[:, 20:23]
    question_Q4.columns = ["Q4.1", "Q4.2", "Q4.3"]
    likert_Q4 = responses.iloc[:, 23:25]
    likert_Q4.columns = ["Q4_Con", "Q4_Sat"]

    question_Q5 = responses.iloc[:, 25:27]
    question_Q5.columns = ["Q5.1", "Q5.2"]
    likert_Q5 = responses.iloc[:, 27:]
    likert_Q5.columns = ["Q5_Con", "Q5_Sat"]

    # Calculate score for each question using calculate function
    score_Q1 = calculate(question_Q1, answer_Q1, "Q1")
    score_Q2 = calculate(question_Q2, answer_Q2, "Q2")
    score_Q3 = calculate(question_Q3, answer_Q3, "Q3")
    score_Q4 = calculate(question_Q4, answer_Q4, "Q4")
    score_Q5 = calculate(question_Q5, answer_Q5, "Q5")

    # Combine into one file
    total = pd.concat(
        [
            score_Q1,
            likert_Q1.reset_index(drop=True),
            score_Q2,
            likert_Q2.reset_index(drop=True),
            score_Q3,
            likert_Q3.reset_index(drop=True),
            score_Q4,
            likert_Q4.reset_index(drop=True),
            score_Q5,
            likert_Q5.reset_index(drop=True),
        ],
        axis=1,
    )

    total["Complexity"] = complexity

    return total

In [1446]:
# answer_Q1 = answer_sheet.iloc[:, 0:3]
# answer_Q2 = answer_sheet.iloc[:, 3:9]
# answer_Q3 = answer_sheet.iloc[:, 9:14]
# answer_Q4 = answer_sheet.iloc[:, 14:17]
# answer_Q5 = answer_sheet.iloc[:, 17:]

In [1447]:
# question_Q1 = complex_responses.iloc[:, 0:3]
# question_Q1.columns = ["Q1.1", "Q1.2", "Q1.3"]
# likert_Q1 = complex_responses.iloc[:, 3:5]
# likert_Q1.columns = ["Q1_Con", "Q1_Sat"]

# question_Q2 = complex_responses.iloc[:, 5:11]
# question_Q2.columns = ["Q2.1", "Q2.2", "Q2.3", "Q2.4", "Q2.5", "Q2.6"]
# likert_Q2 = complex_responses.iloc[:, 11:13]
# likert_Q2.columns = ["Q2_Con", "Q2_Sat"]

# question_Q3 = complex_responses.iloc[:, 13:18]
# question_Q3.columns = ["Q3.1", "Q3.2", "Q3.3", "Q3.4", "Q3.5"]
# likert_Q3 = complex_responses.iloc[:, 18:20]
# likert_Q3.columns = ["Q3_Con", "Q3_Sat"]

# question_Q4 = complex_responses.iloc[:, 20:23]
# question_Q4.columns = ["Q4.1", "Q4.2", "Q4.3"]
# likert_Q4 = complex_responses.iloc[:, 23:25]
# likert_Q4.columns = ["Q4_Con", "Q4_Sat"]

# question_Q5 = complex_responses.iloc[:, 25:27]
# question_Q5.columns = ["Q5.1", "Q5.2"]
# likert_Q5 = complex_responses.iloc[:, 27:]
# likert_Q5.columns = ["Q5_Con", "Q5_Sat"]

In [1448]:
# score_Q1 = calculate(question_Q1, answer_Q1, "Q1")
# score_Q2 = calculate(question_Q2, answer_Q2, "Q2")
# score_Q3 = calculate(question_Q3, answer_Q3, "Q3")
# score_Q4 = calculate(question_Q4, answer_Q4, "Q4")
# score_Q5 = calculate(question_Q5, answer_Q5, "Q5")

In [1449]:
# total = pd.concat(
#     [
#         score_Q1,
#         likert_Q1.reset_index(drop=True),
#         score_Q2,
#         likert_Q2.reset_index(drop=True),
#         score_Q3,
#         likert_Q3.reset_index(drop=True),
#         score_Q4,
#         likert_Q4.reset_index(drop=True),
#         score_Q5,
#         likert_Q5.reset_index(drop=True),
#     ],
#     axis=1,
# )

# total["Complexity"] = "Complex"

In [47]:
complex_df = calculate_scores(complex_responses, answer_sheet, "Complex")
simple_df = calculate_scores(simple_responses, answer_sheet, "Simple")


all_responses = pd.concat(
    [complex_df.reset_index(drop=True), simple_df.reset_index(drop=True)]
)

In [48]:
simple_df.head(20)

,Q1_Accuracy,Q1_Con,Q1_Sat,Q2_Accuracy,Q2_Con,Q2_Sat,Q3_Accuracy,Q3_Con,Q3_Sat,Q4_Accuracy,Q4_Con,Q4_Sat,Q5_Accuracy,Q5_Con,Q5_Sat,Complexity
0,1.000000,3.0,3.0,0.500000,4.0,4.0,0.6,4.0,4.0,0.333333,5.0,3.0,0.0,NaN,NaN,Simple
1,0.333333,5.0,2.0,0.500000,1.0,4.0,0.6,4.0,5.0,0.333333,3.0,2.0,0.0,1.0,1.0,Simple
2,1.000000,5.0,2.0,0.333333,3.0,3.0,0.6,4.0,1.0,0.333333,5.0,3.0,1.0,5.0,3.0,Simple
3,0.666667,5.0,3.0,0.000000,3.0,3.0,0.6,4.0,5.0,0.333333,5.0,4.0,1.0,5.0,4.0,Simple
4,1.000000,5.0,4.0,0.500000,5.0,2.0,0.6,4.0,1.0,0.000000,5.0,4.0,0.5,3.0,2.0,Simple
5,0.666667,1.0,2.0,0.500000,5.0,4.0,0.6,4.0,5.0,0.333333,2.0,3.0,1.0,4.0,5.0,Simple
6,0.333333,2.0,2.0,0.666667,3.0,2.0,0.6,5.0,2.0,0.000000,1.0,1.0,1.0,1.0,2.0,Simple
7,1.000000,5.0,4.0,0.500000,2.0,2.0,0.6,5.0,4.0,0.000000,5.0,2.0,0.5,3.0,2.0,Simple
8,1.000000,5.0,3.0,0.500000,2.0,2.0,0.6,3.0,3.0,0.333333,5.0,4.0,1.0,5.0,4.0,Simple
9,1.000000,5.0,3.0,0.500000,1.0,1.0,0.6,5.0,3.0,0.333333,5.0,2.0,1.0,4.0,4.0,Simple


In [49]:
all_responses.to_csv("./data/transformed/all_responses.csv")

In [50]:
# Calculate User confidence for simple visualizations
simple_responses["UC"] = (
    simple_responses["Q73"]
    + simple_responses["Q76"]
    + simple_responses["Q78"]
    + simple_responses["Q74"]
    + simple_responses["Q80"]
) / 25
# Calculate User satisfaction for simple visualization
simple_responses["US"] = (
    simple_responses["Q72"]
    + simple_responses["Q75"]
    + simple_responses["Q77"]
    + simple_responses["Q79"]
    + simple_responses["Q81"]
) / 25

# Calculate User confidence for complex visualizations
complex_responses["UC"] = (
    complex_responses["QUC1"]
    + complex_responses["Q61"]
    + complex_responses["Q63"]
    + complex_responses["Q65"]
    + complex_responses["Q67"]
) / 25
# Calculate User satisfaction for complex visualization
complex_responses["US"] = (
    complex_responses["QUS1"]
    + complex_responses["Q62"]
    + complex_responses["Q64"]
    + complex_responses["Q66"]
    + complex_responses["Q68"]
) / 25

# show result for simple and complex results
complex_responses[["US", "UC"]]

,US,UC
19,0.48,0.36
24,0.40,0.36
27,0.60,0.52
30,0.80,0.88
31,NaN,NaN
37,0.64,0.72
40,0.68,0.68
47,0.80,0.80


In [51]:
simple_responses[["US", "UC"]]

,US,UC
20,NaN,NaN
21,0.56,0.56
22,0.48,0.88
23,0.76,0.88
25,0.52,0.88
26,0.76,0.64
28,0.36,0.48
29,0.56,0.80
32,0.64,0.80
34,0.52,0.80
